In [1]:
pip install pydicom

In [2]:
import numpy as np 
import pandas as pd
import pydicom
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
import os

from tqdm import tqdm_notebook

In [3]:
classInfo = pd.read_csv('rsna-pneumonia-detection-challenge/stage_2_detailed_class_info.csv')
trainlabels = pd.read_csv("rsna-pneumonia-detection-challenge/stage_2_train_labels.csv")
trainImagesPath = Path("rsna-pneumonia-detection-challenge/stage_2_train_images")
testImagesPath = Path("rsna-pneumonia-detection-challenge/stage_2_test_images")

In [4]:
classInfo.sample(5)

,patientId,class
1383,15bba9aa-1a49-4633-ac86-e8000f676d78,No Lung Opacity / Not Normal
14463,8eef29e3-b9de-4c4f-b1f5-efb9fe7b349a,Lung Opacity
11580,7820e3bb-b904-4d1f-ab4a-2822209e62ee,Normal
13487,8782c8e0-8592-47e0-95df-ee8224d6dec8,Lung Opacity
23737,d58bd53a-c69c-443e-851d-479a78d1c9b2,Normal


In [5]:
trainlabels.head()

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


In [6]:
trainlabels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30227 entries, 0 to 30226
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   patientId  30227 non-null  object 
 1   x          9555 non-null   float64
 2   y          9555 non-null   float64
 3   width      9555 non-null   float64
 4   height     9555 non-null   float64
 5   Target     30227 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 1.4+ MB


In [7]:
classInfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30227 entries, 0 to 30226
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   patientId  30227 non-null  object
 1   class      30227 non-null  object
dtypes: object(2)
memory usage: 472.4+ KB


In [8]:
trainlabels.values

array([['0004cfab-14fd-4e49-80ba-63a80b6bddd6', nan, nan, nan, nan, 0],
       ['00313ee0-9eaa-42f4-b0ab-c148ed3241cd', nan, nan, nan, nan, 0],
       ['00322d4d-1c29-4943-afc9-b6754be640eb', nan, nan, nan, nan, 0],
       ...,
       ['c1f6b555-2eb1-4231-98f6-50a963976431', nan, nan, nan, nan, 0],
       ['c1f7889a-9ea9-4acb-b64c-b737c929599a', 570.0, 393.0, 261.0,
        345.0, 1],
       ['c1f7889a-9ea9-4acb-b64c-b737c929599a', 233.0, 424.0, 201.0,
        356.0, 1]], dtype=object)

In [9]:
classInfo.values

array([['0004cfab-14fd-4e49-80ba-63a80b6bddd6',
        'No Lung Opacity / Not Normal'],
       ['00313ee0-9eaa-42f4-b0ab-c148ed3241cd',
        'No Lung Opacity / Not Normal'],
       ['00322d4d-1c29-4943-afc9-b6754be640eb',
        'No Lung Opacity / Not Normal'],
       ...,
       ['c1f6b555-2eb1-4231-98f6-50a963976431', 'Normal'],
       ['c1f7889a-9ea9-4acb-b64c-b737c929599a', 'Lung Opacity'],
       ['c1f7889a-9ea9-4acb-b64c-b737c929599a', 'Lung Opacity']],
      dtype=object)

In [10]:
trainImagesPath

WindowsPath('rsna-pneumonia-detection-challenge/stage_2_train_images')

# Data Preparation

In [11]:
from skimage.transform import resize

In [12]:
#converting dcm file to array
def image_convert(path):
    ds = pydicom.read_file(path) # read dicom image from filepath 
    image = ds.pixel_array
    image_resized = resize(image, (256,256), anti_aliasing=True)
    return image_resized

In [13]:
test = [image_convert('rsna-pneumonia-detection-challenge/stage_2_train_images/c3b05294-29be-46e4-8a51-96fd211e4ca5.dcm'),image_convert('rsna-pneumonia-detection-challenge/stage_2_train_images/06550cfd-e9d5-415e-bae9-3146329ed9e6.dcm')]

In [14]:
test

[array([[3.34881235e-01, 2.29634009e-01, 1.43972142e-01, ...,
         7.61776202e-02, 1.12042473e-01, 1.39346859e-01],
        [3.25929591e-01, 2.21982300e-01, 1.39414672e-01, ...,
         8.99974456e-03, 3.05687964e-02, 5.93674987e-02],
        [3.21334670e-01, 2.16071620e-01, 1.36464786e-01, ...,
         1.53758453e-04, 1.02549272e-03, 6.48280026e-03],
        ...,
        [1.66752311e-01, 1.69381210e-01, 1.75731141e-01, ...,
         4.52183167e-01, 4.99482808e-01, 5.45647347e-01],
        [1.73333534e-01, 1.75414556e-01, 1.85742531e-01, ...,
         4.57591549e-01, 5.00057565e-01, 5.47358278e-01],
        [1.82568618e-01, 1.90110754e-01, 1.97600104e-01, ...,
         4.58836947e-01, 5.06228675e-01, 5.52820788e-01]]),
 array([[0.08371574, 0.07498948, 0.07424123, ..., 0.76948315, 0.77486628,
         0.78106233],
        [0.07293132, 0.07357513, 0.07285987, ..., 0.76722355, 0.7732537 ,
         0.77758872],
        [0.07182149, 0.07204285, 0.07098172, ..., 0.76483606, 0.77020642,

In [15]:
np.shape(test)

(2, 256, 256)

In [16]:
image_array=[]
image_labels=trainlabels[['patientId','Target']]

In [17]:
images = trainImagesPath.glob('*.dcm')
for image in images:
    image_array.append(image_convert(image))

In [18]:
image_array

[array([[0.34152529, 0.19988844, 0.11420281, ..., 0.31708438, 0.49270066,
         0.60935387],
        [0.13913074, 0.06289526, 0.02733532, ..., 0.10797323, 0.25560768,
         0.44684985],
        [0.03954385, 0.01582917, 0.00892937, ..., 0.0209641 , 0.08566487,
         0.22055383],
        ...,
        [0.02341307, 0.01314812, 0.01190319, ..., 0.0059165 , 0.01914088,
         0.03386861],
        [0.03271006, 0.01719394, 0.01287018, ..., 0.01016684, 0.02640824,
         0.04172344],
        [0.04800501, 0.02419412, 0.01622459, ..., 0.01795929, 0.03836494,
         0.05587304]]),
 array([[0.00000000e+00, 0.00000000e+00, 4.10033479e-08, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 2.82906119e-08, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 9.77423496e-11, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e

In [19]:
np.shape(image_array[0])

(256, 256)

In [20]:
from sklearn.model_selection import train_test_split

# Extracting Metadata

In [21]:
def get_dicom_fps(dicom_dir):
    dicom_fps = glob.glob(dicom_dir+'/'+'*.dcm')
    return list(set(dicom_fps))

def parse_dataset(dicom_dir, anns): 
    image_fps = get_dicom_fps(dicom_dir)
    image_annotations = {fp: [] for fp in image_fps}
    for index, row in anns.iterrows(): 
        fp = os.path.join(dicom_dir, row['patientId']+'.dcm')
        image_annotations[fp].append(row)
    return image_fps, image_annotations 

In [22]:
anns = pd.DataFrame(image_labels)

In [23]:
import glob

In [24]:
image_labels

,patientId,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,0
4,00436515-870c-4b36-a041-de91049b9ab4,1
...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,1
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,0
30224,c1f6b555-2eb1-4231-98f6-50a963976431,0
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,1


In [25]:
image_fps, image_annotations = parse_dataset("rsna-pneumonia-detection-challenge/stage_2_train_images", anns=anns)

In [26]:
image_annotations

{'rsna-pneumonia-detection-challenge/stage_2_train_images\\7127c6c6-261a-4d76-bb98-fabb8595a6ea.dcm': [patientId    7127c6c6-261a-4d76-bb98-fabb8595a6ea
  Target                                          0
  Name: 10693, dtype: object],
 'rsna-pneumonia-detection-challenge/stage_2_train_images\\d9c69b69-38f7-4b21-b43c-95dff9cc94b4.dcm': [patientId    d9c69b69-38f7-4b21-b43c-95dff9cc94b4
  Target                                          0
  Name: 24239, dtype: object],
 'rsna-pneumonia-detection-challenge/stage_2_train_images\\a0a7949a-d305-40f8-a8a4-3cfcb773f3df.dcm': [patientId    a0a7949a-d305-40f8-a8a4-3cfcb773f3df
  Target                                          1
  Name: 16628, dtype: object],
 'rsna-pneumonia-detection-challenge/stage_2_train_images\\88e30597-cf5a-4f8b-a683-5f166bea1cc1.dcm': [patientId    88e30597-cf5a-4f8b-a683-5f166bea1cc1
  Target                                          0
  Name: 13656, dtype: object],
 'rsna-pneumonia-detection-challenge/stage_2_train_image

In [27]:
image_ann =pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in image_annotations.items() ]))

In [28]:
image_ann

,rsna-pneumonia-detection-challenge/stage_2_train_images\7127c6c6-261a-4d76-bb98-fabb8595a6ea.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\d9c69b69-38f7-4b21-b43c-95dff9cc94b4.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\a0a7949a-d305-40f8-a8a4-3cfcb773f3df.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\88e30597-cf5a-4f8b-a683-5f166bea1cc1.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\8a18544c-834f-407c-94bd-4b0d028cd0a5.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\ce76e195-c9ba-4669-ae41-d81fe95979e2.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\f3a3590f-55d7-4da6-80bc-2cb7736e8777.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\f0c03e9e-a3b5-4f45-aa8d-c341c16c50f4.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\3c4c732b-8281-41ef-bd8c-98f98a1c11fe.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\759f55b1-8bd3-4695-a4ac-6ad6336abaed.dcm,...,rsna-pneumonia-detection-challenge/stage_2_train_images\f2bc6d71-2f5b-49ed-941c-2cd7068a83de.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\b8fbe64b-f29f-4c6e-9b4e-999d69719ab8.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\dafe4318-b7bc-4766-9d29-8d4d6d12e670.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\a5759426-c3cc-407f-8abb-8c4314d85caa.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\fbcef55b-b6d2-427b-912c-0e98bb9fc128.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\87c9db93-c4b1-4488-9f8d-d94ea473973c.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\917980f4-ad9e-4bc1-96f2-0786f11a8a85.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\ecfdcc29-e54e-4565-ab0f-24d2c73df79e.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\9cbf14f3-d3dc-418d-a149-c0b859655dbb.dcm,rsna-pneumonia-detection-challenge/stage_2_train_images\5885c03d-7eb8-4472-8ad1-646e7d41fd48.dcm
0,patientId 7127c6c6-261a-4d76-bb98-fabb8595a...,patientId d9c69b69-38f7-4b21-b43c-95dff9cc9...,patientId a0a7949a-d305-40f8-a8a4-3cfcb773f...,patientId 88e30597-cf5a-4f8b-a683-5f166bea1...,patientId 8a18544c-834f-407c-94bd-4b0d028cd...,patientId ce76e195-c9ba-4669-ae41-d81fe9597...,patientId f3a3590f-55d7-4da6-80bc-2cb7736e8...,patientId f0c03e9e-a3b5-4f45-aa8d-c341c16c5...,patientId 3c4c732b-8281-41ef-bd8c-98f98a1c1...,patientId 759f55b1-8bd3-4695-a4ac-6ad6336ab...,...,patientId f2bc6d71-2f5b-49ed-941c-2cd7068a8...,patientId b8fbe64b-f29f-4c6e-9b4e-999d69719...,patientId dafe4318-b7bc-4766-9d29-8d4d6d12e...,patientId a5759426-c3cc-407f-8abb-8c4314d85...,patientId fbcef55b-b6d2-427b-912c-0e98bb9fc...,patientId 87c9db93-c4b1-4488-9f8d-d94ea4739...,patientId 917980f4-ad9e-4bc1-96f2-0786f11a8...,patientId ecfdcc29-e54e-4565-ab0f-24d2c73df...,patientId 9cbf14f3-d3dc-418d-a149-c0b859655...,patientId 5885c03d-7eb8-4472-8ad1-646e7d41f...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,patientId b8fbe64b-f29f-4c6e-9b4e-999d69719...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
image_fps

['rsna-pneumonia-detection-challenge/stage_2_train_images\\7127c6c6-261a-4d76-bb98-fabb8595a6ea.dcm',
 'rsna-pneumonia-detection-challenge/stage_2_train_images\\d9c69b69-38f7-4b21-b43c-95dff9cc94b4.dcm',
 'rsna-pneumonia-detection-challenge/stage_2_train_images\\a0a7949a-d305-40f8-a8a4-3cfcb773f3df.dcm',
 'rsna-pneumonia-detection-challenge/stage_2_train_images\\88e30597-cf5a-4f8b-a683-5f166bea1cc1.dcm',
 'rsna-pneumonia-detection-challenge/stage_2_train_images\\8a18544c-834f-407c-94bd-4b0d028cd0a5.dcm',
 'rsna-pneumonia-detection-challenge/stage_2_train_images\\ce76e195-c9ba-4669-ae41-d81fe95979e2.dcm',
 'rsna-pneumonia-detection-challenge/stage_2_train_images\\f3a3590f-55d7-4da6-80bc-2cb7736e8777.dcm',
 'rsna-pneumonia-detection-challenge/stage_2_train_images\\f0c03e9e-a3b5-4f45-aa8d-c341c16c50f4.dcm',
 'rsna-pneumonia-detection-challenge/stage_2_train_images\\3c4c732b-8281-41ef-bd8c-98f98a1c11fe.dcm',
 'rsna-pneumonia-detection-challenge/stage_2_train_images\\759f55b1-8bd3-4695-a4ac

In [30]:
len('17af8e70-3af0-4a47-a5ba-f16938c87c7e.dcm')

40

In [31]:
image_fps_list = []
for i in image_fps:
    image_fps_list.append(i[-40:-4])

In [32]:
sorted(image_fps_list)

['0004cfab-14fd-4e49-80ba-63a80b6bddd6',
 '000924cf-0f8d-42bd-9158-1af53881a557',
 '000db696-cf54-4385-b10b-6b16fbb3f985',
 '000fe35a-2649-43d4-b027-e67796d412e0',
 '001031d9-f904-4a23-b3e5-2c088acd19c6',
 '0010f549-b242-4e94-87a8-57d79de215fc',
 '001916b8-3d30-4935-a5d1-8eaddb1646cd',
 '0022073f-cec8-42ec-ab5f-bc2314649235',
 '0022995a-45eb-4cfa-9a59-cd15f5196c64',
 '0025d2de-bd78-4d36-9f72-e15a5e22ca82',
 '00293de0-a530-41dc-9621-0b3def01d06d',
 '002c591d-df62-4e34-8eda-838c664430a9',
 '002cb550-2e31-42f1-a29d-fbc279977e71',
 '00313ee0-9eaa-42f4-b0ab-c148ed3241cd',
 '00322d4d-1c29-4943-afc9-b6754be640eb',
 '003d8fa0-6bf1-40ed-b54c-ac657f8495c5',
 '00436515-870c-4b36-a041-de91049b9ab4',
 '0049f7b2-229b-4a46-91a3-0e321a3ca80a',
 '0053ee32-1e03-4dc2-a103-f46ad0448787',
 '00569f44-917d-4c86-a842-81832af98c30',
 '005f5517-ab76-4b0d-8a26-f808d97171c4',
 '0067d70b-0e6d-41ff-a9b8-8edf6d5e25e4',
 '006cec2e-6ce2-4549-bffa-eadfcd1e9970',
 '006e75c8-1fd9-4a5a-99e7-285addebed55',
 '00704310-78a8-

In [33]:
a = list(image_labels['patientId'])

In [34]:
len(set(a)),len(set(image_fps_list))

(26684, 26684)

In [35]:
image_labels.drop_duplicates(keep='last', inplace = True)

C:\Users\u1137734\AppData\Local\Temp\ipykernel_16548\2833128290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  image_labels.drop_duplicates(keep='last', inplace = True)


In [36]:
image_labels.shape

(26684, 2)

In [37]:
np.shape(image_fps)

(26684,)

In [38]:
import warnings
warnings.filterwarnings('ignore')

In [39]:
image_labels.set_index('patientId', inplace = True)

In [40]:
X = image_array
y = image_labels

In [41]:
image_labels.head()

,Target
patientId,
0004cfab-14fd-4e49-80ba-63a80b6bddd6,0
00313ee0-9eaa-42f4-b0ab-c148ed3241cd,0
00322d4d-1c29-4943-afc9-b6754be640eb,0
003d8fa0-6bf1-40ed-b54c-ac657f8495c5,0
00436515-870c-4b36-a041-de91049b9ab4,1


In [42]:
np.shape(y)

(26684, 1)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [44]:
len(X_train), len(y_train), len(X_test), len(y_test)

(18678, 18678, 8006, 8006)

# Model creation and Training

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
import tensorflow.keras.layers as layers

In [46]:
'''
model = Sequential()
model.add(Flatten(input_shape = [256,256]))
model.add(Dense(200, activation = 'relu'))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(1, activation = 'softmax'))
'''

"\nmodel = Sequential()\nmodel.add(Flatten(input_shape = [256,256]))\nmodel.add(Dense(200, activation = 'relu'))\nmodel.add(Dense(50, activation = 'relu'))\nmodel.add(Dense(1, activation = 'softmax'))\n"

In [47]:
model = Sequential()
model.add(layers.Conv2D(256, (10, 10), activation='relu', input_shape=(256,256,1)))
model.add(layers.MaxPooling2D((10, 10)))
model.add(layers.Conv2D(256, (10, 10), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1,activation = 'sigmoid'))

In [48]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 247, 247, 256)     25856     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 256)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 256)       6553856   
                                                                 
 flatten (Flatten)           (None, 57600)             0         
                                                                 
 dense (Dense)               (None, 64)                3686464   
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                        

In [49]:
import tensorflow as tf

In [50]:
type(tf.stack(X_train))

tensorflow.python.framework.ops.EagerTensor

In [51]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer = 'adam', metrics = 'accuracy')
model.fit(tf.stack(X_train), y_train,validation_data=(X_test, y_test)) #try np.array once

584/584 [==============================] - ETA: 0s - loss: 0.5548 - accuracy: 0.7751

ValueError: Data cardinality is ambiguous:
  x sizes: 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 2

In [ ]:
y_predict = model.predict(tf.stack(X_test))

In [ ]:
model.evaluate(tf.stack(X_test), y_test)

In [ ]:
model.save_weights('simple_model.h5')

In [56]:
my_image = pydicom.dcmread('C:\\Users\\u1137734\\Downloads\\rsna-pneumonia-detection-challenge\\stage_2_train_images\\00aecb01-a116-45a2-956c-08d2fa55433f.dcm')
    #Step 2
# my_image_re = resize(my_image, (256,256,3))
cv2.imwrite('tempfile.png',my_image.pixel_array)
my_image = cv2.imread('tempfile.png')
my_image_re = np.array(cv2.resize(my_image,dsize = (256,256))).reshape(-1,256,256,1)

probabilities = model.predict(my_image_re)
print(probabilities)

1/1 [==============================] - 0s 133ms/step
(3, 1)
